<a href="https://colab.research.google.com/github/Tulipraaj/FoCID-Forecasting_Cryptocurrency_Investment_Decision/blob/main/Bi_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import math as m
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error ,r2_score

In [2]:
df = pd.read_csv("BTC.csv")

In [3]:
df.head()

,date,block_size,difficulty,hashrate,market_cap,mining_profitability,transaction,tweets,price
0,01-01-2018,966169,1.922580e+12,1.500000e+19,2.266610e+11,2.174,241601,62044.0,13523
1,02-01-2018,968332,1.931140e+12,1.600000e+19,2.334150e+11,2.431,340809,77723.0,13924
2,03-01-2018,975882,1.931140e+12,1.490000e+19,2.519290e+11,2.737,395806,79086.0,15027
3,04-01-2018,975713,1.931140e+12,1.640000e+19,2.495520e+11,2.606,424840,74534.0,14883
4,05-01-2018,974894,1.931140e+12,1.500000e+19,2.663230e+11,2.631,342564,76404.0,15881


In [4]:

df.tail()

,date,block_size,difficulty,hashrate,market_cap,mining_profitability,transaction,tweets,price
1821,27-12-2022,706675,3.536410e+13,2.120000e+20,3.234390e+11,0.0589,239244,85632.0,16807
1822,28-12-2022,542347,3.536410e+13,2.530000e+20,3.202390e+11,0.0704,272781,107117.0,16640
1823,29-12-2022,692913,3.536410e+13,2.740000e+20,3.193230e+11,0.0545,265814,96225.0,16591
1824,30-12-2022,725552,3.536410e+13,2.540000e+20,3.185390e+11,0.0658,290857,97462.0,16550
1825,31-12-2022,573296,3.536410e+13,2.940000e+20,3.189440e+11,0.0557,242169,77382.0,16570


In [5]:
df.isna().sum()

date                     0
block_size               0
difficulty               0
hashrate                 0
market_cap               0
mining_profitability     0
transaction              0
tweets                  37
price                    0
dtype: int64

In [6]:
df.dtypes

date                     object
block_size                int64
difficulty              float64
hashrate                float64
market_cap              float64
mining_profitability    float64
transaction               int64
tweets                  float64
price                     int64
dtype: object

In [7]:
df.fillna(method='ffill', inplace=True)


In [8]:
df.isna().sum()

date                    0
block_size              0
difficulty              0
hashrate                0
market_cap              0
mining_profitability    0
transaction             0
tweets                  0
price                   0
dtype: int64

In [9]:
X = df[['tweets','market_cap','difficulty','hashrate','mining_profitability']]
# Assuming 'Close' is the target variable
y = df['price']

In [10]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(np.array(y).reshape(-1, 1))


In [11]:
def create_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        seq = data[i:i + sequence_length]
        sequences.append(seq)
    return np.array(sequences)

In [12]:
sequence_length = 15  # Adjust this based on how many past data points you want to use for prediction
X_sequence = create_sequences(X_scaled, sequence_length)
y_sequence = y_scaled[sequence_length:]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_sequence, y_sequence, test_size=0.2, random_state=42)

MODEL 1


In [14]:
model = Sequential()
model.add(Bidirectional(LSTM(20, activation='relu', input_shape=(sequence_length, X_train.shape[2]),return_sequences=True)))
model.add(Bidirectional(LSTM(20, activation='relu', input_shape=(sequence_length, X_train.shape[2]))))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse', 'mae'])

In [16]:
batch_sizes = [40, 60, 80, 100, 120]

def mean_percentage_error(y_true, y_pred):
    return np.mean((y_true - y_pred) / y_true) * 100

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

for batch_size in batch_sizes:
    print(f"Training with batch size: {batch_size}")
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse', 'mae'])
    model.fit(X_train, y_train,epochs=200, batch_size=batch_size, validation_split=0.1)

    y_pred = model.predict(X_test)

    y_pred_original = scaler.inverse_transform(y_pred)
    y_test_original = scaler.inverse_transform(y_test)
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_test)

    mse = mean_squared_error(y_test_original, y_pred_original)
    mae = mean_absolute_error(y_test_original, y_pred_original)
    mpe = mean_percentage_error(y_test_original, y_pred_original)
    mape = mean_absolute_percentage_error(y_test_original, y_pred_original)
    r2 = r2_score(y_train, y_train_pred)

    print(f"Metrics for batch size {batch_size}:")
    print("Mean Squared Error:", mse)
    print("Root Mean Squared Error:", m.sqrt(mse))
    print("Mean Absolute Error:", mae)
    print("Mean Percentage Error:", mpe)
    print("Mean Absolute Percentage Error:", mape)
    print("R2_Score:",r2)
    print(model.summary())
    print("-" * 40)

Training with batch size: 40
Epoch 1/200
33/33 [==============================] - 5s 37ms/step - loss: 2.6750e-04 - mse: 2.6750e-04 - mae: 0.0107 - val_loss: 1.7069e-04 - val_mse: 1.7069e-04 - val_mae: 0.0080
Epoch 2/200
33/33 [==============================] - 1s 21ms/step - loss: 1.6068e-04 - mse: 1.6068e-04 - mae: 0.0078 - val_loss: 1.7242e-04 - val_mse: 1.7242e-04 - val_mae: 0.0083
Epoch 3/200
33/33 [==============================] - 1s 25ms/step - loss: 1.6033e-04 - mse: 1.6033e-04 - mae: 0.0078 - val_loss: 1.8768e-04 - val_mse: 1.8768e-04 - val_mae: 0.0086
Epoch 4/200
33/33 [==============================] - 1s 22ms/step - loss: 1.6072e-04 - mse: 1.6072e-04 - mae: 0.0078 - val_loss: 2.0128e-04 - val_mse: 2.0128e-04 - val_mae: 0.0089
Epoch 5/200
33/33 [==============================] - 0s 15ms/step - loss: 1.6481e-04 - mse: 1.6481e-04 - mae: 0.0080 - val_loss: 2.7997e-04 - val_mse: 2.7997e-04 - val_mae: 0.0104
Epoch 6/200
33/33 [==============================] - 0s 15ms/step - los